# 2D DDW
## Pedicting Crop Type from data based in India



In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import process, fuzz

### **Model Outline(stuff yall can follow):**
> **Warning:** DO NOT MANIPULATE THE CSV FILES DIRECTLY. Any manipulation will be done by loading the data into a variable in your code. CSV FILES SHOULD BE UNTOUCHED. However you can create and export new CSV Files based on original data.


#### Step 1 Data Preprocessing:

**CSV Files**
```bash
├── Fertilizer.csv
├── Crop_Yield.csv
```
- Use charts to visualize, summarize and understand your data. Whatever stuff you guys can think of that will help someone to understand the data better(I'll do this~Gangesh)
- Display both files using pandas and take a look at the columns. You need to find a way to combine this datasets and remove some of the unnecessary columns(ex. Production column in final dataset. Crops which are not meant for consumptions like cotton,jute etc). 
- **Note:** Some of the crop names in the soil conditions file and the Crop Yield file dont match exactly. Find a way to accurately still combine the datasets.
- After which, recall what i said about data that are not numbers, in this case states. Look up panda dummy variables.
- Normalize the data. Can use the one in problems set.
  
#### Step 2 Training and Validation:
- Follow the initial steps exactly as in cohort 9 to train your first model
- evaluate the model using an appropriate metric. Indicate all data of the trained model together with the error through your chosen error function. Visualize the trained model/training of the model with maybe types of graph
- Take steps to improve your model. Try to search online. One way is to vary the iterations and learning rate. Train a few times, then choose the model with the best accuracy.

#### Step 3 Prediction:
- Find a way to predict the crop type that gives the highest yield. 
- Recall that the above model predicts highest yield for any given crop type. So what you would do is create a function where the user types in everything except for crop, and you would go on to predict yield for every crop with the inputted variables. After which you will choose the crop with the highest yield and return it 
- In the crop yield csv you will notice a crop_type column. This refers the the type of seasonal crop they are. Ex. Rabi Crops are harvested in the spring season while it is sown in winter. Create a function where if i input a date, I can receive the corresponding crop type.


### Date  Pre-processing

In [115]:
df_TempCropYield = pd.read_csv("Crop_Yield.csv")
df_CropSoilConditions = pd.read_csv("Crop_Ideal_Soil_Conditions.csv")
#add soil condition columns to start combining
df_TempCropYield['N'] = 0
df_TempCropYield['P'] = 0
df_TempCropYield['K'] = 0
df_TempCropYield['pH'] = 0

# string1 = "Ragi( naachnnii)"
# string2 = "Ragi"
# str1 = "Chili"
# str2 = "Dry chillies"
# # Calculate the similarity score using fuzz.ratio
# score1 = fuzz.partial_ratio(string1,string2)
# score2 = fuzz.partial_ratio(str1,str2)
# print(score1)
# print(score2)
# Using partial_ratio returned the best comparison result on sample set

# Delete rows with cotton or jute in the 'Crop' column as those are not edible
values_to_delete = ['Cotton(lint)', 'Jute']
df_TempCropYield = df_TempCropYield[~df_TempCropYield['Crop'].isin(values_to_delete)]
# Mapping dictionary for crop name conversion to assist in combining datasets
crop_mapping = {
    'Arhar/Tur': 'Pigeon peas(Toor Dal)',
    'Moong(Green Gram)': 'Mung beans',
    'Urad': 'Black Gram',
    'Sesamum': 'Sesame Seed',
    'Bhindi': 'Lady Finger',
    'Cowpea(Lobia)': 'Black eyed beans(Chawli)',
    'Guar seed': 'Cluster Beans(Gavar)'
}
# Replace crop names in the 'Crop' column
df_TempCropYield['Crop'] = df_TempCropYield['Crop'].replace(crop_mapping)
display(df_TempCropYield)

for index, row in df_TempCropYield.iterrows():
    # Match the crop string using fuzzywuzzy
    match = process.extractOne(row['Crop'], df_CropSoilConditions['Crop'],scorer=fuzz.partial_ratio)
    
    # Randomly pick one of the matches
    matched_rows = df_CropSoilConditions[df_CropSoilConditions['Crop'] == match[0]]
    random_matched_row = matched_rows.sample(n=1)
    
    # Copy the values to df_TempCropYield if a match is found
    if match[1] >= 80:  # Replace YOUR_THRESHOLD with your desired matching threshold
        df_TempCropYield.at[index, 'N'] = random_matched_row['N'].values[0]
        df_TempCropYield.at[index, 'P'] = random_matched_row['P'].values[0]
        df_TempCropYield.at[index, 'K'] = random_matched_row['K'].values[0]
        df_TempCropYield.at[index, 'pH'] = random_matched_row['pH'].values[0]
    else:
        # Add NaN if no match is found
        df_TempCropYield.at[index, 'N'] = None
        df_TempCropYield.at[index, 'P'] = None
        df_TempCropYield.at[index, 'K'] = None
        df_TempCropYield.at[index, 'pH'] = None
#Drop crows for which soil data cant be found
df_TempCropYield = df_TempCropYield.dropna(subset=['N'])
df_TempCropYield.drop(['Area_in_hectares', 'Production_in_tons'], axis=1)
#convert categories to dummy variables
df_TempCropYield_NoCat = pd.get_dummies(df_TempCropYield,dtype=int)
#move y column to last
df_TempCropYield_NoCat["Yield_ton_per_hec"] = df_TempCropYield_NoCat.pop("Yield_ton_per_hec")
df_TempCropYield_NoCat.to_csv('Crop_Yield_Combined.csv', index=False)


State_Name   Crop_Type                   Crop  rainfall  \
0       andhra pradesh      kharif  Pigeon peas(Toor Dal)    654.34   
1       andhra pradesh      kharif                  Bajra    654.34   
2       andhra pradesh      kharif            Castor seed    654.34   
4       andhra pradesh      kharif           Dry chillies    654.34   
5       andhra pradesh      kharif              Groundnut    654.34   
...                ...         ...                    ...       ...   
173021     west bengal      summer                  Maize    182.50   
173022     west bengal      summer                   Rice    182.50   
173023     west bengal  whole year              Sugarcane   1501.98   
173024     west bengal        rabi                   Rice    152.54   
173025     west bengal        rabi            Sesame Seed    152.54   

        temperature  Area_in_hectares  Production_in_tons  Yield_ton_per_hec  \
0         29.266667           21400.0              2600.0           0.121495   
1         29.266667            1400.0               500.0           0.357143   
2         29.266667            1000.0               100.0           0.100000   
4         29.266667            3700.0              7100.0           1.918919   
5         29.266667          650800.0            228400.0           0.350953   
...             ...               ...                 ...                ...   
173021    29.200000             258.0               391.0           1.515504   
173022    29.200000             105.0               281.0           2.676190   
173023    25.818182             303.0             11541.0          38.089109   
173024    22.280000          152676.0            261435.0           1.712352   
173025    22.280000             244.0                95.0           0.389344   

        N  P  K  pH  
0       0  0  0   0  
1       0  0  0   0  
2       0  0  0   0  
4       0  0  0   0  
5       0  0  0   0  
...    .. .. ..  ..  
173021  0  0  0   0  
173022  0  0  0   0  
173023  0  0  0   0  
173024  0  0  0   0  
173025  0  0  0   0  

[168811 rows x 12 columns]

/var/folders/k5/kwnm67cj11j95x28sdwr66dw0000gn/T/ipykernel_3632/1162020465.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.54' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_TempCropYield.at[index, 'pH'] = random_matched_row['pH'].values[0]
